Before staring working with groq, register on the website and receive your own unike API key. 

In [1]:
from groq_api import KEY
from groq import Groq
import numpy as np
import pandas as pd
from tqdm import tqdm

import json

CLIENT = Groq(api_key=KEY)
PATH = 'data/power/power-ua-train.tsv'


Read csv file as a pandas dataframe, set index to the index from the csv file, add "prediction" and "reasoning" columns.
Lastly, shaffle the data (by sampling all the rows in random order), setting random state for reproducibility.

In [2]:
data = pd.read_csv(PATH, sep='\t', index_col=0)
# set object column dtype full of nans
data['prediction'] = np.nan
data['reasoning'] = np.nan
data['labeled_prediction'] = np.nan
data = data.sample(frac=1, random_state=42)
data.head()


,speaker,sex,text,text_en,label,prediction,reasoning,labeled_prediction
id,,,,,,,,
ua05796,d18023ad9b6df26ee266bb01fc4c822a,M,"Олександр Шевченко, фракція ""<PARTY>"". Ратифік...","Alexander Shevchenko, Freedom faction. Ratific...",1,NaN,NaN,NaN
ua07362,bea4118a3082afea430ffe03d4343b76,M,"Шановний пане Голово, шановні колеги, дозвольт...","Dear Mr. Head, honourable colleagues, let me i...",0,NaN,NaN,NaN
ua10560,b07c8d09c03a6a3222057c300c84e9fe,M,"Олег Купрієнко, <PARTY>. А ще порядок в Україн...","Oleg Kuprienko, Oleg Laska Radical Party. And ...",1,NaN,NaN,NaN
ua10327,93a1d11f680ed07b734319a5fae11cee,F,"По-перше, що стосується військового госпіталю,...","First of all, it's about a military hospital i...",0,NaN,NaN,NaN
ua15066,e7d36da025052d2213a757d55deeb1d1,M,Шановна президія! Шановні народні депутати! Ре...,Honorable Predestination! Dear National Deputi...,0,NaN,NaN,NaN


In [11]:
print('Size of data: ', data.shape[0])

Size of data:  11324


In order to get examples for a few-shot learning, get a few inccorrectly classified answers from LLama and then provide the reasoning in those answers as a prompt to a model (I got those ids from groq_ua_few and made analysis outside of this notebook.)

In [3]:
curated_ids = ['ua05929', 'ua14412', 'ua15441', 'ua10563']
data.loc['ua05929', 'prediction'] = 'opposition'
data.loc['ua14412', 'prediction'] = 'coalition'
data.loc['ua15441', 'prediction'] = 'coalition'
data.loc['ua10563', 'prediction'] = 'opposition'

data.loc['ua05929', 'reasoning'] = 'The speaker is trying to persuade the majority to support Euro-integration laws and is emphasizing the importance of resolving key issues, including the release of Yulia Timoshenko. Since the speaker is opposing the "majority", it suggests that they are in the minority, in the opposition.'
data.loc['ua14412', 'reasoning'] = 'The speaker addresses a minister "with respect", the tone of his plea is heartful. He asks the minister to protect the Luhansk oblast from another bureacrat Mr. Pshik. Since he asks so politely, wants to be protected, and addresses the minister — a person in power — it suggests the speaker is from coalition party.'
data.loc['ua15441', 'reasoning'] = 'The speaker addresses others as "colleagues", the tone of his speech is neutral, respectful. Therefore, he belongs to coalition.'
data.loc['ua10563', 'reasoning'] = 'The speaker is proposing amendments to a bill and providing detailed explanations and justifications for their proposals. The tone is professional but also critical to the suggested bill: the speakes says the bill contains contradictions, as well as that the bill interprets some things ""too narrowly"". Because of this critisism, the speaker belongs to opposition.'

/var/folders/tg/w5th396s1r146pfch3q10j180000gn/T/ipykernel_45700/1467728001.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'opposition' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc['ua05929', 'prediction'] = 'opposition'
/var/folders/tg/w5th396s1r146pfch3q10j180000gn/T/ipykernel_45700/1467728001.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'The speaker is trying to persuade the majority to support Euro-integration laws and is emphasizing the importance of resolving key issues, including the release of Yulia Timoshenko. Since the speaker is opposing the "majority", it suggests that they are in the minority, in the opposition.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc['ua05929', 'reasoning'] = 'The speaker is trying to persu

Filter out data of curated_ids from the dataset into a new dataframe. Add a column "json_response" to it with the curated reasoning and a correct label in order to send this json_response to the model in the next steps.

In [20]:
few_shots_data = data[data.index.isin(curated_ids)]
few_shots_data.loc[:, "json_response"] = few_shots_data.apply(lambda x: f'{{"reasoning": "{x.reasoning}", "prediction": "{x.prediction}"}}', axis=1)
few_shots_data

/var/folders/tg/w5th396s1r146pfch3q10j180000gn/T/ipykernel_45700/2989662672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  few_shots_data.loc[:, "json_response"] = few_shots_data.apply(lambda x: f'{{"reasoning": "{x.reasoning}", "prediction": "{x.prediction}"}}', axis=1)


,speaker,sex,text,text_en,label,prediction,reasoning,labeled_prediction,json_response
id,,,,,,,,,
ua05929,79de6e68e0ea3d3c52105d1122207a45,M,"Звертаюсь до представників більшості, питання ...","I'm talking to the majority, the question to e...",1,opposition,The speaker is trying to persuade the majority...,NaN,"{""reasoning"": ""The speaker is trying to persua..."
ua15441,0934150a731ecd1d56efc59cb5d95974,M,"Колеги, коли навіть Железняк помітив, що потрі...","Colleagues, when even the Ironman noticed the ...",0,coalition,"The speaker addresses others as ""colleagues"", ...",NaN,"{""reasoning"": ""The speaker addresses others as..."
ua14412,cda3f314dd19759c84327a32fe87f5b3,M,"Дякую. Денис Анатолійович, вітаємо вас! І з по...","Thank you. Denise Anatolyovich, welcome! And w...",0,coalition,"The speaker addresses a minister ""with respect...",NaN,"{""reasoning"": ""The speaker addresses a ministe..."
ua10563,885573d8848fa398173c06dc22668f8d,M,"Семенуха Роман, фракція ""Об'єднання ""<PARTY>"",...","Semenha Roman, the Self-Help Union faction, a ...",1,opposition,The speaker is proposing amendments to a bill ...,NaN,"{""reasoning"": ""The speaker is proposing amendm..."


Here, I am defining a function which sends calls to Llama3 using groq API and uses as examples of the answers on behalf of the model my curated answers above.

In [ ]:
SYSTEM_PROMPT = '''You are an expert in Ukrainian political life and parlamentary debates in the 2012-2024 years. Help me identify whether this speech was held by coalition or opposition. First reason about the text and try to find helpful cues. Then provide a final answer. Formulate your response in JSON using the following format: {"reasoning": "...", "prediction": "coalition or opposition"}. Make sure to return proper JSON, e.g. don't add superfluous single quotation marks in the end.'''

In [14]:
def get_response_few_shot(text, few_shots_data):
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
    ]
    for _, row in few_shots_data.iterrows():
        messages.append({"role": "user", "content": row.text})
        messages.append({"role": "assistant", "content": row.json_response})

    messages.append({"role": "user", "content": text})
    
    for i in range(5):
        try:
            # in a try loop, 
            # I am increasing temperature to get a response in case of an error generating JSON
            chat_completion = CLIENT.chat.completions.create(
                messages=messages,
                model="llama3-70b-8192",
                response_format={"type": "json_object"},
                temperature=0.2 * i,             
            )
            response_content = chat_completion.choices[0].message.content
            response = json.loads(response_content)
            return response
        except Exception as e:
            print(e)
            continue
    else:
        return None


In [15]:
NUM_SAMPLES = 670   # in order to compare with 0-shot learning
# NUM_SAMPLES = int(data.shape[0] * 0.2)  # 20% of the data
error_counter = 0

for index, entry in (tqdm_bar := tqdm(data.iloc[:NUM_SAMPLES].iterrows(), total=NUM_SAMPLES)):
    text = entry['text']
    response = get_response_few_shot(text, few_shots_data)
    if not response:
        error_counter += 1
        continue
    if response['prediction'] not in ['coalition', 'opposition']:
        error_counter += 1
        continue
    data.loc[index, 'prediction'] = response['prediction']
    data.loc[index, 'reasoning'] = response['reasoning']
    data.loc[index, 'labeled_prediction'] = int(data.loc[index, 'prediction'] == 'opposition')

    # Update true/false count to see in the progress bar 
    data_subset = data.loc[:index].dropna(subset=['labeled_prediction', 'prediction'])
    distribution = (data_subset['labeled_prediction'] == data_subset['label']).value_counts(normalize=True)
    tqdm_bar.set_description(f"Accuracy: {distribution.get(True, 0):.2f}")

  0%|          | 0/670 [00:00<?, ?it/s]

Accuracy: 0.86:   1%|          | 7/670 [00:15<25:16,  2.29s/it]

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{"reasoning": "The speaker is criticizing the Committee on European Integration for not following the rules and not providing conclusions to the deputies. The tone is critical and slightly emotional, with phrases like ""frankly, it\'s not clear"" and ""pure violation"". This suggests that the speaker is from the opposition.", "prediction": "opposition"}'}}


Accuracy: 0.78:   1%|▏         | 9/670 [00:29<47:55,  4.35s/it]  

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{"reasoning": "The speaker is criticizing the Ukrainian government and Foreign Ministry for not doing enough to liberate Ukrainian political prisoners in Russia. The tone is emotional and patriotic, with phrases like ""Glory to Ukraine! The glory of the nation!"". The speaker also mentions that only the Radical Party faction in Parliament supports their cause, implying that they are in the opposition.", "prediction": "opposition"}'}}
Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{"reasoning": "The speaker is criticizing the Ukrainian government and Foreign Ministry for not doing enough to li

...

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{"reasoning": "The speaker is calling for support for a draft decision, which was developed by a party and signed by all faction heads. The tone is emotional and passionate, with strong criticism towards Russia and Putin. The speaker is using strong language, calling Russia a ""terrorist state"". This kind of rhetoric is typical for opposition politicians, who often use strong language to criticize the government or other countries.", "prediction": "opposition"}'}}


Accuracy: 0.71: 100%|█████████▉| 669/670 [1:01:24<00:04,  4.22s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hxy93jbdfyjv7ffybcry6r6c` on tokens per minute (TPM): Limit 6000, Used 0, Requested ~6112. Please try again in 1.12s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hxy93jbdfyjv7ffybcry6r6c` on tokens per minute (TPM): Limit 6000, Used 0, Requested ~6112. Please try again in 1.12s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hxy93jbdfyjv7ffybcry6r6c` on tokens per minute (TPM): Limit 6000, Used 0, Requested ~6112. Please try again in 1.12s. Visit https://console.groq.com/docs/rate-limits for more information.

Accuracy: 0.71: 100%|██████████| 670/670 [1:02:17<00:00,  5.58s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hxy93jbdfyjv7ffybcry6r6c` on tokens per minute (TPM): Limit 6000, Used 0, Requested ~6112. Please try again in 1.12s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [18]:
data_subset = data.dropna(subset=['labeled_prediction', 'prediction'])
distribution = (data_subset['labeled_prediction'] == data_subset['label']).value_counts(normalize=True)
print(f"Accuracy: {distribution.get(True, 0):.2f}")
print(f'Errors:', error_counter)

Accuracy: 0.71
Errors: 6


In [20]:
# save the results into a csv file
import os 

FOLDER_PATH = 'results'

if not os.path.exists(FOLDER_PATH):
    os.makedirs(FOLDER_PATH)

data.to_csv('results/power-ua-results-curated.tsv', sep='\t')